# Imports

In [5]:
import pm4py
import pandas as pd
from TreeToTptp import TreeToTptp
from ProverWrapper import use_provers
from create_thesis_with_2_spec import create_thesis_with_2_spec, LogicFunctions
import os

In [6]:
def print_provers(provers):
    print(provers[0])
    print(f'Vampire: {provers[1]}')
    print(f'Eprover: {provers[2]}')


def provers_to_csv(provers):
    print(','.join(provers))

In [7]:
def create_problem_folder(folder_name, expr, spec, tptp):
    os.makedirs(folder_name, exist_ok=True)
    with open(f'{folder_name}/expression.txt', 'w') as f:
        f.write(expr)
    with open(f'{folder_name}/specification.txt', 'w') as f:
        f.write(spec)
    with open(f'{folder_name}/tptp.p', 'w') as f:
        f.write(tptp)
    vampire, eprover = use_provers(f'{folder_name}/tptp.p')
    with open(f'{folder_name}/vampire.txt', 'w') as f:
        f.write(vampire)
    with open(f'{folder_name}/eprover.txt', 'w') as f:
        f.write(eprover)

# def create_problem_folder_with_2_spec(folder_name, expr1, spec1, tptp1, expr2, spec2, tptp2, ):
    

# Julia's problems

In [8]:
log = pm4py.read_xes('Data/running-example.xes')

process_tree = pm4py.discover_process_tree_inductive(
    log, 0.0, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
expr, spec, tptp = TreeToTptp().tree_to_tptp(process_tree)
create_problem_folder("test", expr, spec, tptp)

parsing log, completed traces :: 100%|██████████| 6/6 [00:00<00:00, 1553.16it/s]

Seq3(1]register_request, Loop(2]l_s_0, Seq2(3]And2(4]a2_s_0, check_ticket, Xor2(5]x2_s_0, examine_thoroughly, examine_casually, x2_e_0[5), a2_e_0[4), decide[3), reinitiate_request[2), Xor2(2]x2_s_1, reject_request, pay_compensation, x2_e_1[2)[1)
ini: register_request
fin: x2_e_1



/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/conversion/log/variants/to_data_frame.py:73: UserWarning: no explicit representation of timezones available for np.datetime64
  ev[attr] = np.datetime64(ev[attr])
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/d